In [1]:
import pandas as pd

In [2]:
# %store -r freak_df
# %store -r Lore_df
# %store -r Invisible_df
%store -r tal_df
# %store -r OnBeing_df
%store -r radiolab_df
%store -r uu_df
# %store -r StoryCorps_df
%store -r YWA_df
%store -r myDNA_df
%store -r Nightvale_df
# read in maxfun data then store

In [3]:
# create a multindexed dataframe
data = pd.concat([Nightvale_df,
                  myDNA_df,
                  YWA_df,
                  # StoryCorps_df,
                  uu_df,
                  radiolab_df,
                  # OnBeing_df,
                  # Invisible_df,
                  # Lore_df,
                  tal_df],
                 keys = ['Nightvale','Move Your DNA','You\'re Wrong About','Unlocking Us','Radiolab','This American Life'])

In [4]:
data

Episode  Year                               Title  \
Nightvale          88     000a  2018              matter of blood part 2   
                   11      001  2012                               pilot   
                   12      002  2012                          glow cloud   
                   40      003  2012                  station management   
                   14      004  2012                         pta meeting   
...                        ...   ...                                 ...   
This American Life 686     730   NaN                     The Empty Chair   
                   685     731   NaN                   What Lies Beneath   
                   689     732   NaN                             Secrets   
                   688     733   NaN              Warriors in the Garden   
                   0       734   NaN  The Campus Tour Has Been Cancelled   

                                                                     Text  
Nightvale          88   CECIL: As a matter of fact, the facts don’t ma...  
                   11   And now the news. Old Woman Josie, out near th...  
                   12   And now, the news. Have any of our listeners s...  
                   40   The Night Vale Business Association is proud t...  
                   14   The Pteranodons mostly attacked women with gla...  
...                                                                   ...  
This American Life 686     Prologue: Prologue        Announcer   A qui...  
                   685     Prologue: Prologue        Announcer   A qui...  
                   689     Prologue: Prologue        Susan Burton   It...  
                   688     Prologue: Prologue        Announcer   A qui...  
                   0       Prologue: Prologue        Ira Glass   OK, L...  

[1302 rows x 4 columns]

I don't yet have the data from maximum fun (working on web-scraping embedded pdfs), I can start analysis here.  I'm going to start with basic text stats like the ones from homework 2 - token count, word length, sentence length, etc.